Importes necesarios

In [1]:
import polars as pl
import os
import zipfile
import orjson as json
import tqdm as tqdm # Usamos .notebook para barras de carga bonitas en Jupyter

# --- CONFIGURACIÓN ---
# Ajusta estas rutas a tus carpetas reales.
# Si estás en Windows y usas backslashes (\), usa r"./data/KR"
CARPETA_DATA = r"./Data/"

DATA_PATHS = {
    "EUW": CARPETA_DATA + r"matches_raw_euw_ranked.zip",
    "KR": CARPETA_DATA + r"matches_raw_kr_ranked.zip",
    "NA": CARPETA_DATA + r"matches_raw_na_ranked.zip"
}
OUTPUT_FILE = "draft_oracle_master_data.parquet"

# Verificación rápida: ¿Existen las carpetas?
print(f"Directorio de trabajo actual: {os.getcwd()}")
for region, path in DATA_PATHS.items():
    exists = os.path.exists(path)
    status = "✅ Encontrada" if exists else "❌ NO ENCONTRADA (Revisa la ruta)"
    print(f"Región {region} ({path}): {status}")

Directorio de trabajo actual: D:\Proyectos\SkyTheLimit
Región EUW (./Data/matches_raw_euw_ranked.zip): ✅ Encontrada
Región KR (./Data/matches_raw_kr_ranked.zip): ✅ Encontrada
Región NA (./Data/matches_raw_na_ranked.zip): ✅ Encontrada


Definición de la Función de Extracción

In [2]:
def extract_features_complete(data, region, filename="unknown"):
    """
    Combina las métricas Base (v1) y Pro (v2) en un solo registro.
    """
    try:
        info = data.get('info', {})
        meta = data.get('metadata', {})

        # Filtros
        if info.get('queueId', 0) != 420 or info.get('gameDuration', 0) < 900:
            return []

        match_id = meta.get('matchId', filename)

        extracted_rows = []
        for p in info.get('participants', []):
            challenges = p.get('challenges', {})
            perks = p.get('perks', {})

            # Cálculo DCR
            dmg = p.get('totalDamageDealtToChampions', 0)
            gold = p.get('goldEarned', 1)
            dcr = dmg / gold if gold > 0 else 0

            # Runas
            try:
                styles = perks.get('styles', [])
                primary = styles[0].get('style', -1) if styles else -1
                keystone = styles[0].get('selections', [])[0].get('perk', -1) if styles and styles[0].get('selections') else -1
            except:
                primary = -1; keystone = -1

            row = {
                # --- IDENTIFICADORES ---
                "game_id": match_id, "region": region, "patch": info.get('gameVersion', "0.0.0"),
                "champ_id": p.get('championId'), "position": p.get('teamPosition', 'UTILITY'),
                "side": p.get('teamId'), "target": 1 if p.get('win') else 0, "duration": info.get('gameDuration', 0), "queue": info.get('queueId', 0),

                # --- 1. PERFIL DE DAÑO (BASE) ---
                "stat_phys_dmg": p.get('physicalDamageDealtToChampions', 0),
                "stat_magic_dmg": p.get('magicDamageDealtToChampions', 0),
                "stat_true_dmg": p.get('trueDamageDealtToChampions', 0),
                "stat_dmg_taken": p.get('totalDamageTaken', 0),
                "stat_mitigated": p.get('damageSelfMitigated', 0),
                "stat_heal": p.get('totalHealsOnTeammates', 0),

                # --- 2. PERFIL DE UTILIDAD (BASE + PRO) ---
                "stat_cc_duration": p.get('timeCCingOthers', 0),
                "stat_hard_cc": challenges.get('enemyChampionImmobilizations', 0),
                "stat_vision": challenges.get('visionScorePerMinute', 0),

                # --- 3. ECONOMÍA Y MACRO (PRO) ---
                "stat_dcr": dcr, # Nuevo
                "stat_dpm": challenges.get('damagePerMinute', 0),
                "stat_gpm": challenges.get('goldPerMinute', 0),
                "stat_turret_plates": challenges.get('turretPlatesTaken', 0),
                "stat_obj_control": challenges.get('dragonTakedowns', 0),

                # --- 4. LANE & ROAMING (PRO) ---
                "stat_lane_cs_10": challenges.get('laneMinionsFirst10Minutes', 0), # Faltaba este!
                "stat_solo_kills": challenges.get('soloKills', 0),
                "stat_lane_diff": challenges.get('laningPhaseGoldExpAdvantage', 0),
                "stat_roam_kills": challenges.get('killsOnOtherLanesEarlyJungleAsLaner', 0),
                "stat_dodge": challenges.get('skillshotsDodged', 0),

                # --- TÁCTICAS ---
                "rune_primary": primary, "rune_keystone": keystone
            }
            extracted_rows.append(row)
        return extracted_rows
    except: return []

In [3]:
# --- PARÁMETROS DE RENDIMIENTO ---
BATCH_SIZE = 5000  # Convertir a Polars cada 5000 partidas para liberar RAM
chunks_list = []   # Lista de DataFrames optimizados
current_batch = [] # Lista temporal de filas

print("🚀 Iniciando Ingesta (Soporte para Archivos ZIP Directos)...")

def flush_batch():
    if not current_batch: return
    try:
        df_chunk = pl.DataFrame(current_batch)
        # Casting Masivo de todas las columnas 'stat_' a Float32 para ahorrar RAM
        # Detectamos dinámicamente las columnas que empiezan por 'stat_'
        stat_cols = [c for c in df_chunk.columns if c.startswith("stat_")]

        # Schema de optimización
        optimizations = [
            pl.col("game_id").cast(pl.Utf8),
            pl.col("region").cast(pl.Categorical),
            pl.col("position").cast(pl.Categorical),
            pl.col("patch").cast(pl.Utf8),
            pl.col("target").cast(pl.Int8),
            pl.col("side").cast(pl.Int16),
            pl.col("champ_id").cast(pl.Int16),
            pl.col("rune_primary").cast(pl.Int16),
            pl.col("rune_keystone").cast(pl.Int16),
            pl.col("duration").cast(pl.Int16),
            pl.col("queue").cast(pl.Int16)
        ] + [pl.col(c).cast(pl.Float32) for c in stat_cols] # Todo lo stat_ a float

        df_chunk = df_chunk.with_columns(optimizations)
        chunks_list.append(df_chunk)
    except Exception as e:
        print(f"⚠️ Error chunk: {e}")
    current_batch.clear()

# --- BUCLE PRINCIPAL ---
for region, path in DATA_PATHS.items():
    if not os.path.exists(path):
        print(f"❌ Ruta no encontrada: {path}")
        continue

    print(f"📂 Procesando Región {region} en: {path}")

    # LÓGICA HÍBRIDA: ¿Es un archivo ZIP o una Carpeta?

    # CASO A: Es un archivo ZIP directo (Tu configuración actual)
    if os.path.isfile(path) and path.endswith(".zip"):
        try:
            with zipfile.ZipFile(path, "r") as z:
                all_json_names = [f for f in z.namelist() if f.endswith(".json")]

                # Barra de carga para los archivos DENTRO del zip
                for json_name in tqdm.tqdm(all_json_names, desc=f"📦 Extrayendo {region}", unit="json"):
                    with z.open(json_name) as f:
                        try:
                            # Leemos y parseamos
                            content = f.read()
                            data = json.loads(content)

                            # Extraemos features
                            rows = extract_features_complete(data, region, filename=json_name)
                            current_batch.extend(rows)

                            # Batch flush
                            if len(current_batch) >= BATCH_SIZE * 10:
                                flush_batch()
                        except Exception:
                            continue
        except Exception as e:
            print(f"❌ Error crítico leyendo el ZIP {path}: {e}")

    # CASO B: Es una Carpeta (Configuración antigua)
    elif os.path.isdir(path):
        files_in_folder = os.listdir(path)
        for filename in tqdm.tqdm(files_in_folder, desc=f"📂 Carpeta {region}"):
            full_path = os.path.join(path, filename)

            # (Reutilizamos la lógica de zip/json aquí si fuera necesario,
            # pero tu error indica que estás en el Caso A)
            if filename.endswith(".json"):
                 try:
                    with open(full_path, "rb") as f: # rb para compatibilidad con orjson
                        data = json.loads(f.read())
                        rows = extract_features_complete(data, region, filename=filename)
                        current_batch.extend(rows)
                        if len(current_batch) >= BATCH_SIZE * 10:
                            flush_batch()
                 except: continue

# Procesar el último remanente
flush_batch()

print(f"✅ Ingesta finalizada. Se generaron {len(chunks_list)} chunks optimizados.")

🚀 Iniciando Ingesta (Soporte para Archivos ZIP Directos)...
📂 Procesando Región EUW en: ./Data/matches_raw_euw_ranked.zip


📦 Extrayendo EUW: 100%|██████████| 210238/210238 [01:50<00:00, 1906.59json/s]


📂 Procesando Región KR en: ./Data/matches_raw_kr_ranked.zip


📦 Extrayendo KR: 100%|██████████| 61381/61381 [00:32<00:00, 1886.76json/s]


📂 Procesando Región NA en: ./Data/matches_raw_na_ranked.zip


📦 Extrayendo NA: 100%|██████████| 69393/69393 [00:36<00:00, 1922.16json/s]

✅ Ingesta finalizada. Se generaron 59 chunks optimizados.


In [4]:
if len(chunks_list) > 0:
    print("🧩 Unificando chunks en un solo DataFrame Maestro...")

    # concat es muy eficiente porque los chunks ya tienen el mismo esquema y tipos
    try:
        df_master = pl.concat(chunks_list)

        print(f"💾 Guardando {len(df_master)} registros en {OUTPUT_FILE}...")
        df_master.write_parquet(OUTPUT_FILE, compression="snappy")

        print(f"🎉 ¡ETL Completado! Dataset Final: {df_master.shape}")

        # Liberar memoria de la lista de chunks
        del chunks_list

    except Exception as e:
        print(f"❌ Error al concatenar: {e}. Verifica que todos los chunks tengan las mismas columnas.")
else:
    print("⚠️ No se extrajeron datos.")

🧩 Unificando chunks en un solo DataFrame Maestro...
💾 Guardando 2928910 registros en draft_oracle_master_data.parquet...
🎉 ¡ETL Completado! Dataset Final: (2928910, 30)
